In [4]:
:dep plotters = { version = "^0.3.5", default_features = true, features = ["evcxr", "all_series", "all_elements"] }
:dep rstrading = { version = "*", path="../" }
:dep csv = { version = "*" }
:dep serde = { version = "1.0.143", features = ["derive"] }
:dep chrono = { version = "0.4.26" }

extern crate plotters;
use plotters::prelude::*;

extern crate rstrading;
extern crate csv;
use std::fs::File;
use serde::{Deserialize}; 

use rstrading::data::Candle;
use chrono::{DateTime, TimeZone, NaiveDateTime, Utc};

In [5]:
/// Load the json data into variables

const DATA_HISTORIC_XAUUSD_CANDLES_1M: &str = "./candles/xauusd_1m.csv";

#[derive(Deserialize)]
struct Record {
    ticker: String,
    date: String,
    time: String,
    open: f64,
    high: f64,
    low: f64,
    close: f64,
    vol: i32,
}

fn load_json_market_event_candles() -> (Vec<Candle>, DateTime<Utc>, DateTime<Utc>, f64, f64) {
    let file = File::open(DATA_HISTORIC_XAUUSD_CANDLES_1M).unwrap();
    let mut rdr = csv::ReaderBuilder::new().has_headers(true).from_reader(file);
    let mut candles = Vec::new();
    
    let mut xMin = chrono::DateTime::<Utc>::MAX_UTC;
    let mut xMax = chrono::DateTime::<Utc>::MIN_UTC;
    
    let mut yMin = f64::MAX;
    let mut yMax = f64::MIN;
    
    for result in rdr.deserialize::<Record>() {
        let record = result.unwrap();
        let date_str = record.date + " " + record.time.as_str() + " +00:00";
        let close_time = DateTime::parse_from_str(&date_str, "%Y%m%d %H%M%S %z").unwrap().with_timezone(&Utc);
        //let close_time_millis = close_time;
        candles.push(Candle{
            close_time,
            open: record.open,
            high: record.high,
            low: record.low,
            close: record.close,
            volume: Some(record.vol as f64),
            trade_count: None,
        });
        
        if (close_time > xMax) {
            xMax = close_time;
        }
        
        if (close_time < xMin) {
            xMin = close_time;
        }
        
        if (record.high > yMax) {
            yMax = record.high;
        }
        
        if (record.low < yMin) {
            yMin = record.low;
        }
    }
    
    (candles, xMin, xMax, yMin, yMax)
}

let (candles, xMin, xMax, yMin, yMax) = load_json_market_event_candles();
candles[0..3]

[Candle { close_time: 2022-06-08T00:00:00Z, open: 1849.0, high: 1849.1, low: 1849.0, close: 1849.0, volume: Some(4.0), trade_count: None }, Candle { close_time: 2022-06-08T00:01:00Z, open: 1849.1, high: 1849.3, low: 1849.0, close: 1849.3, volume: Some(4.0), trade_count: None }, Candle { close_time: 2022-06-08T00:02:00Z, open: 1849.4, high: 1849.4, low: 1849.2, close: 1849.3, volume: Some(4.0), trade_count: None }]

In [13]:
// Draw the chart
let chart_width = 1280;
evcxr_figure((chart_width, 400), |root| {
    root.fill(&WHITE)?;
//     let start_date = Utc.with_ymd_and_hms(2022, 06, 08, 0, 0, 0).unwrap();
//     let end_date = Utc.with_ymd_and_hms(2022, 06, 08, 23, 41, 00).unwrap();
    let mut chart = ChartBuilder::on(&root)
        .x_label_area_size(60)
        .y_label_area_size(60)
        .build_cartesian_2d(xMin..xMax, yMin..yMax)
//     .build_cartesian_2d(start_date..end_date, yMin..yMax)
        .unwrap();
    
    chart
        .configure_mesh()
        .light_line_style(&WHITE)
        .draw()
        .unwrap();
    
    let candleWidth = (chart_width as usize) / candles.len();
    chart.draw_series(candles.iter().map(|x| {
        CandleStick::new(
            x.close_time,
            x.open,
            x.high,
            x.low,
            x.close,
            &GREEN,
            &RED,
            candleWidth as u32,
        )
    })).unwrap();
    Ok(())
})

1846.0
 
 
 
1847.0
 
 
 
1848.0
 
 
 
1849.0
 
 
 
1850.0
 
 
 
1851.0
 
 
 
 
2022-06-08T00:00:00Z
 
 
 
2022-06-08T01:00:00Z
 
 
 
2022-06-08T02:00:00Z
 
 
 
2022-06-08T03:00:00Z
 
 
 
2022-06-08T04:00:00Z
 
 
 
2022-06-08T05:00:00Z
 
 
 
2022-06-08T06:00:00Z
 
 
 
2022-06-08T07:00:00Z
 
 
 
2022-06-08T08:00:00Z